# 电商移动推荐算法竞赛实践


比赛入口

* https://tianchi.aliyun.com/competition/entrance/231522/introduction


本课件参考了以下链接中的想法，代码为重新编写
* https://blog.csdn.net/Datuqiqi/article/details/46834579
* https://blog.csdn.net/Snoopy_Yuan/article/category/6924508
* https://github.com/chenkkkk/TianChi_YiDongTuiJian_forecast 



本课件数据下载地址，此数据做了缩短，原来一个月的数据为了方便缩短为12-10到12-18日，如果需要全量数据请自行参加比赛下载，数据说明和之前的天猫品牌行为数据类似，behavior_type分为1浏览，2收藏，3购物车，4购买，多了经纬度数据user_geohash但是没用。
* https://pan.baidu.com/s/13SrbmDWGBqQDlnfqB5yI1w


重要代码

* me.accuracy_score(y_true, y_pred)
* me.recall_score(y_true, y_pred)
* me.f1_score(y_true, y_pred)
* me.precision_score(y_true, y_pred)

## 推荐的原理与特征处理

#### 数据读取

https://www.bilibili.com/video/BV1s7411w7Xv?p=17

#### groupby？

https://www.bilibili.com/video/BV1Z5411L7vU?p=9


#### unstack

https://blog.csdn.net/qq_42874547/article/details/89056000

#### 数据处理三板斧——map、apply、applymap详解

https://zhuanlan.zhihu.com/p/100064394

In [2]:
import sklearn.metrics as me
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier #公认这里比较好的算法，我们就直接用了，而且这个方法不太需要特征标准化   
from sklearn.model_selection import train_test_split,GridSearchCV 

* 行为类型（behavior_type）：以每天的浏览，收藏，购物车，购买行为数量构建准特征

In [3]:
u_i_table=pd.read_csv(r"D:\jupyterDate\python_data_course\data_course\机器学习与深度学习导论\comp_offline_data\tianchi_user2.csv",parse_dates=[5])
i_table=pd.read_csv(r"D:\jupyterDate\python_data_course\data_course\机器学习与深度学习导论\comp_offline_data\tianchi_fresh_comp_train_item.csv")
u_i_table = u_i_table[u_i_table.item_id.isin(i_table.item_id.values)]#过滤掉所有不属于预测商品集的商品行为记录

In [5]:
u_i_table

,user_id,item_id,behavior_type,user_geohash,item_category,time
9,10001082,275221686,1,NaN,10576,2014-12-13 14:00:00
15,10001082,125083630,1,NaN,4722,2014-12-14 03:00:00
21,10001082,275221686,1,NaN,10576,2014-12-17 05:00:00
27,10001082,275221686,1,NaN,10576,2014-12-13 14:00:00
46,10001082,275221686,1,NaN,10576,2014-12-13 14:00:00
...,...,...,...,...,...,...
7595696,65186019,232372730,1,NaN,11991,2014-12-16 21:00:00
7595731,65186019,162073666,1,NaN,2334,2014-12-16 21:00:00
7595800,65186019,371822750,1,NaN,11639,2014-12-14 23:00:00
7595901,65186019,65029981,1,NaN,198,2014-12-16 21:00:00


In [14]:
z1=u_i_table.groupby(['user_id','item_id','item_category','behavior_type',pd.Grouper(key='time',freq='1D')]).size()
z1=z1.unstack(-1,fill_value=0).unstack(-1,fill_value=0)


In [15]:
z1

time                              2014-12-10          2014-12-11           \
behavior_type                              1  2  3  4          1  2  3  4   
user_id   item_id   item_category                                           
492       31040941  6986                   0  0  0  0          0  0  0  0   
3726      264960    10412                  0  0  0  0          2  0  0  0   
19137     98921119  3942                   0  0  0  0          0  0  0  0   
38745     38770686  3942                   0  0  0  0          0  0  0  0   
          272985263 3942                   0  0  0  0          0  0  0  0   
...                                      ... .. .. ..        ... .. .. ..   
142442955 260934112 732                    0  0  0  0          0  0  0  0   
          273532679 732                    0  0  0  0          0  0  0  0   
          325938147 732                    0  0  0  0          0  0  0  0   
          364397686 732                    0  0  0  0          0  0  0  0   
          386680968 732                    0  0  0  0          0  0  0  0   

time                              2014-12-12     ... 2014-12-16    2014-12-17  \
behavior_type                              1  2  ...          3  4          1   
user_id   item_id   item_category                ...                            
492       31040941  6986                   0  0  ...          0  0          0   
3726      264960    10412                  0  0  ...          0  0          0   
19137     98921119  3942                   2  0  ...          0  0          0   
38745     38770686  3942                   0  0  ...          0  0          0   
          272985263 3942                   0  0  ...          0  0          0   
...                                      ... ..  ...        ... ..        ...   
142442955 260934112 732                    0  0  ...          0  0          0   
          273532679 732                    0  0  ...          0  0          0   
          325938147 732                    0  0  ...          0  0          0   
          364397686 732                    0  0  ...          0  0          0   
          386680968 732                    0  0  ...          0  0          0   

time                                       2014-12-18           
behavior_type                      2  3  4          1  2  3  4  
user_id   item_id   item_category                               
492       31040941  6986           0  0  0          0  0  0  0  
3726      264960    10412          0  0  0          0  0  0  0  
19137     98921119  3942           0  0  0          0  0  0  0  
38745     38770686  3942           0  0  0          0  0  0  0  
          272985263 3942           0  0  0          0  0  0  0  
...                               .. .. ..        ... .. .. ..  
142442955 260934112 732            0  0  0          0  0  0  0  
          273532679 732            0  0  0          0  0  0  0  
          325938147 732            0  0  0          0  0  0  0  
          364397686 732            0  0  0          0  0  0  0  
          386680968 732            0  0  0          0  0  0  0  

[268137 rows x 36 columns]

In [8]:
len(u_i_table)

682059

## 基于单天行为简单统计的算法

* 基于上面的准特征矩阵构建可用的特征矩阵与目标向量，并通过标准流程完成训练，然后基于精准度对模型进行评价
* 注意这里以16和17的数据为训练集，以17和18的数据为测试集。

* GradientBoostingClassifier()
* map(lambda x:1 if x>0 else 0)
* me.accuracy_score(y_true, y_pred)

In [17]:
x_train=z1["2014-12-16"]
y_train=z1["2014-12-17"][4]
y_train=y_train.map(lambda x:1 if x>0 else 0)

In [19]:
x_test=z1["2014-12-17"]
y_test=z1["2014-12-18"][4]
y_test=y_test.map(lambda x:1 if x>0 else 0)

In [23]:
G0=GradientBoostingClassifier()
G0.fit(x_train.values,y_train.values)
me.accuracy_score(y_test.values,G0.predict(x_test.values),normalize=False)

267406

真实值为0的样本太多，仅依靠准确率的计算不靠谱

## 二分类的模型评价


|  预测与真实 | 真实为正 | 真实为负 |
| ------ | ------ | ------ |
|预测为正 | TP：True Positive,被判定为正样本，事实上也是正样本。 | FP：False Positive,被判定为正样本，但事实上是负样本。 |
| 预测为负| FN：False Negative,被判定为负样本，但事实上是正样本 | TN：True Negative,被判定为负样本，事实上也是负样本 |


* 查准率，所有正预测中正确缺的比率。

$\text{precision} = \frac{tp}{tp + fp}$

* 查全率，所有正样本中被预测到的比率。

$\text{recall} = \frac{tp}{tp + fn}$

* F 值，查准率和查全率的调和平均值。

$F_\beta = (1 + \beta^2) \frac{\text{precision} \times \text{recall}}{\beta^2 \text{precision} + \text{recall}}$

* F 值可以解释为 precision （精度）和 recall （召回）的加权调和平均值。 $F_\beta$ 值达到其最佳值 1 ，其最差分数为 0 。当 $\beta = 1$, $F_\beta$ 和 $F_1$ 是等价的， 此时意味着recall和 precision同样重要，# beta值越小，表示越看中precision，beta值越大，表示越看中recall，不同等重要的场景。

* 以上指标只关注正样本的预测情况。这和很多实际情况是一致的，比如推荐系统，欺诈识别等等
* 不是所有的二分类问题都会使用$F_\beta$
* 二分类问题通常在于正负样本比例非常不平衡

In [26]:
me.f1_score(y_test.values,G0.predict(x_test.values))

0.0

## 样本不均衡问题

* 通过随机选择负样本达到正负均衡

* pd.concat

In [54]:
xy=z1[["2014-12-16","2014-12-17"]].iloc[:,[0,1,2,3,7]]
#xy1=xy[xy.iloc[:,4]==1]
xy.iloc[:,4]=xy.iloc[:,4].map(lambda x:1 if x>0 else 0)
xy1=xy[xy["2014-12-17"][4]==1]
xy0=xy[xy["2014-12-17"][4]==0].sample(frac=0.02)#抽样，因此每次运行结果都不一样
#因为负样本对于模型的训练没有贡献，因此可以减少负样本的数量，以达到正负样本均衡的目的
xy_train=pd.concat([xy1,xy0])
xy_train

time                             2014-12-16          2014-12-17
behavior_type                             1  2  3  4          4
user_id  item_id   item_category                               
1096774  57739007  2284                   4  0  1  0          1
1105255  22302304  732                    0  0  0  0          1
1285111  264035656 13042                  0  0  0  0          1
1287220  148781276 3064                   0  0  0  0          1
1675813  202021439 5273                   0  0  0  0          1
...                                     ... .. .. ..        ...
50845804 177005001 12630                  0  0  0  0          0
58308738 205773530 8254                   0  0  0  0          0
61260896 284090303 2284                   0  0  0  0          0
65098032 84508768  11406                  0  0  0  0          0
63219119 154405197 24                     0  0  0  0          0

[6024 rows x 5 columns]

In [55]:
G0=GradientBoostingClassifier()
G0.fit(xy_train["2014-12-16"].values,xy_train["2014-12-17"][4].values)
me.f1_score(y_test.values,G0.predict(x_test.values))

0.04847950639048039

behavior_type                      1  2  3  4
user_id   item_id   item_category            
492       31040941  6986           0  0  0  0
3726      264960    10412          0  0  0  0
19137     98921119  3942           0  0  0  0
38745     38770686  3942           0  0  0  0
          272985263 3942           0  0  0  0
...                               .. .. .. ..
142442955 260934112 732            0  0  0  0
          273532679 732            0  0  0  0
          325938147 732            0  0  0  0
          364397686 732            0  0  0  0
          386680968 732            0  0  0  0

[268137 rows x 4 columns]

In [52]:
x_test=z1["2014-12-17"]
y_test=z1["2014-12-18"][4]
y_test=y_test.map(lambda x:1 if x>0 else 0)

In [53]:
G0=GradientBoostingClassifier()
G0.fit(xy_train["2014-12-16"].values,xy_train["2014-12-17"][4].values)
me.f1_score(y_test.values,G0.predict(x_test.values))

0.05303370786516854

## 超参数——调参